In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from src.data_processing1 import ModelReadyDataProcessor

In [2]:
import pandas as pd
df = pd.read_csv("../data/raw/data.csv")

X = df.drop(columns=["FraudResult"])
y = df["FraudResult"]


In [3]:
processor = ModelReadyDataProcessor(
    scaling_method="Standard"
)

processor.fit(X, y)
X_model_ready = processor.transform(X)
processor.save_processed(X_model_ready)

X_model_ready.head()


✅ Processed data saved to: D:\AI mastery\credit-risk-model\data\processed\processed.csv


,Value,CurrencyCode_UGX,CountryCode_256,ProviderId_ProviderId_1,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,ProductCategory_woe,ChannelId_woe
0,-0.072291,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.620379,-0.472374
1,-0.080251,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.565446,2.709037
2,-0.076352,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.620379,-0.472374
3,0.096648,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.134963,-0.472374
4,-0.075183,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.565446,2.709037
